In [1]:
import numpy as np
import os
import pickle

from keras.utils import np_utils
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, concatenate
from keras.layers.convolutional import Conv2D, Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adadelta

from matplotlib import pyplot as plt 
import string
from keras.models import load_model



Using TensorFlow backend.


### 配置参数

In [2]:
captcha_word = string.digits + string.ascii_letters
word_len = 5
word_class = len(captcha_word)

width = 200
height = 60


train_dir = 'D:/data/YZM/second_train'
train_label = 'D:/data/YZM/second_train_label.txt'

#生成字符索引
char_indices = dict((c, i) for i,c in enumerate(captcha_word))
indices_char = dict((i, c) for i,c in enumerate(captcha_word))


#验证码字符串转数组
def captcha_to_vec(captcha):    
   
    vector = np.zeros(word_len * word_class)
    for i,ch in enumerate(captcha):
        idex = i * word_class + char_indices[ch]
        vector[idex] = 1
    return vector

#把数组转换回文字
def vec_to_captcha(vec):
    text = []
    vec[vec < 0.5] = 0    
    char_pos = vec.nonzero()[0]
    for i, ch in enumerate(char_pos):
        text.append(captcha_word[ch % word_class])
    return ''.join(text)


In [3]:
image_list = []

for item in os.listdir(train_dir):
    image_list.append(item)


print(len(image_list))

9700


In [4]:

X = np.zeros((len(image_list), height, width, 3), dtype = np.uint8)
y = np.zeros((len(image_list), word_len * word_class), dtype = np.uint8)

with open(train_label) as file_object:
    p = file_object.readlines()
for i,img in enumerate(image_list):
    img_path = train_dir + "/" + img
    raw_img = image.load_img(img_path, target_size=(height, width))
    X[i] = image.img_to_array(raw_img)
    for j in range(len(p)):
        if img[:4]==p[j][:4]:
            break;
    y[i] = captcha_to_vec(p[j][5:10])

print("ok")

ok


In [5]:
#保存成pkl文件
file = open('second_train_data.pkl','wb')
pickle.dump((X,y) , file)
print("ok")

ok


### 加载数据，读取pickle文件

In [3]:
#读取pickle文件
file = open('second_train_data.pkl', 'rb')
X, y = pickle.load(file)
print("ok")

ok


### 创建模型

In [4]:

input_tensor = Input( shape=(height, width, 3))

x = input_tensor

x = Convolution2D(32, 3, padding='same', activation='relu')(x)
x = Convolution2D(32, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)


x = Convolution2D(64, 3, padding='same', activation='relu')(x)
x = Convolution2D(64, 3, padding='same', activation='relu')(x)
x = Convolution2D(128, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)


x = Convolution2D(128, 3, padding='same',activation='relu')(x)
x = Convolution2D(128, 3, padding='same',activation='relu')(x)
x = Convolution2D(256, 3, padding='same', activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.4)(x)

x = [Dense(word_class, activation='softmax', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)

model = Model(inputs=input_tensor, outputs=output)



In [5]:
opt = Adadelta(lr=0.1)
model.compile(loss = 'categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath="D:/data/YZM/weight_2/weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", 
                               verbose=2, save_weights_only=True)
print("ok")

ok


### 开始训练模型

In [9]:
model.fit(X, y, epochs=6,callbacks=[checkpointer],validation_split=0.1)

Train on 8730 samples, validate on 970 samples
Epoch 1/6
8730/8730 [==============================] - 1731s 198ms/step - loss: 8.0702 - acc: 0.9814 - val_loss: 8.0918 - val_acc: 0.9990
Epoch 2/6
8730/8730 [==============================] - 1806s 207ms/step - loss: 8.0677 - acc: 0.9861 - val_loss: 8.0965 - val_acc: 0.9990
Epoch 3/6
 480/8730 [>.............................] - ETA: 34:13 - loss: 8.0635 - acc: 0.9917

KeyboardInterrupt: 

In [11]:
#保存权重和模型
model.save_weights('second_model_weights.h5')
model.save('second_model.h5')